# Titanic dataset
* Get sample data [here](https://github.com/fuyu-quant/IBLM/tree/main/datasets).

In [1]:
%%capture
!pip install git+https://github.com/fuyu-quant/IBLM.git@feature-in-context-learning 

In [2]:
import pkg_resources
print(pkg_resources.get_distribution('IBLM').version)

0.1.45


In [3]:
from dotenv import load_dotenv

# .env ファイルを読み込む
load_dotenv(override=True)

True

In [30]:
import pandas as pd
import numpy as np
from langchain.llms import OpenAI
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from iblm import IBLMClassifier

import sys
import os

In [90]:
n = 50

df = pd.read_csv(f'../data/titanic/titanic_{n}_train.csv')
x_train = df.drop('survived', axis=1)
y_train = df['survived']
print(len(x_train))

50


In [96]:
llm_model = OpenAI(temperature=0, model_name = 'gpt-4-0613')

params = {'columns_name': True}

iblm = IBLMClassifier(llm_model = llm_model, params=params)

### Training

In [92]:
file_path = '../models/titanic/'

model = iblm.fit(x_train, y_train, model_name = 'titanic', file_path=file_path)

Tokens Used: 7444
	Prompt Tokens: 7167
	Completion Tokens: 277
Successful Requests: 1
Total Cost (USD): $0.23162999999999997


In [ ]:
# Code of the model created
print(model)

import numpy as np

def predict(x):
    df = x.copy()
    output = []
    for index, row in df.iterrows():
        # Do not change the code before this point.
        # Please describe the process required to make the prediction below.

        # The logic here is to give higher probability for those who are female, in first class, and embarked from Cherbourg
        # These are based on the known survival facts from the Titanic incident
        # The age, fare, and number of siblings/spouses/parents/children are also considered
        # The weights for each factor are determined based on their perceived impact on survival

        y = 0.3 * row['sex_female'] + 0.2 * row['class_First'] + 0.1 * row['embarked_C'] - 0.1 * row['age']/80 - 0.1 * row['fare']/500 - 0.1 * row['sibsp']/8 - 0.1 * row['parch']/6

        # The result is then passed through a sigmoid function to ensure the output is between 0 and 1
        y = 1 / (1 + np.exp(-y))

        # Do not change the code after this poin

### Prediction

In [81]:
df = pd.read_csv(f'../data/titanic/titanic_{n}_test.csv')
x_test = df.drop('survived', axis=1)
y_test = df['survived']

In [82]:
y_proba = iblm.predict(x_test)
y_pred = (y_proba > 0.5).astype(int)

In [83]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Precision
precision = precision_score(y_test, y_pred)
print(f'Precision: {precision}')

# Recall
recall = recall_score(y_test, y_pred)
print(f'Recall: {recall}')

# F1 score
f1 = f1_score(y_test, y_pred)
print(f'F1 score: {f1}')

# ROC-AUC (you need prediction probabilities for this, not just class predictions)
# Here we just reuse y_pred for simplicity
roc_auc = roc_auc_score(y_test, y_proba)
print(f'ROC-AUC: {roc_auc}')

Accuracy: 0.7164179104477612
Precision: 0.589098532494759
Recall: 0.8463855421686747
F1 score: 0.6946847960444994
ROC-AUC: 0.8317052998636476


### Creating Multiple Models

In [97]:
llm_model = OpenAI(temperature=0, model_name = 'gpt-4-0613')

params = {'columns_name': True}

iblm = IBLMClassifier(llm_model = llm_model, params=params)

In [98]:
seed = 3655
train_data = 300

train_num = [6,8,10,20,30,40,50]


for i in train_num:
    df = pd.read_csv(f'../data/titanic/titanic_{seed}_{train_data}_train.csv').head(i)
    x_train = df.drop('survived', axis=1)
    y_train = df['survived']

    file_path = '../models/titanic/'
    for j in range(1,31):
        model = iblm.fit(x_train, y_train, model_name = f'titanic_{seed}_{i}_{j}', file_path=file_path)

Tokens Used: 1634
	Prompt Tokens: 1343
	Completion Tokens: 291
Successful Requests: 1
Total Cost (USD): $0.057749999999999996


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


Tokens Used: 1659
	Prompt Tokens: 1343
	Completion Tokens: 316
Successful Requests: 1
Total Cost (USD): $0.05925
Tokens Used: 1741
	Prompt Tokens: 1343
	Completion Tokens: 398
Successful Requests: 1
Total Cost (USD): $0.06417
Tokens Used: 1612
	Prompt Tokens: 1343
	Completion Tokens: 269
Successful Requests: 1
Total Cost (USD): $0.05643
Tokens Used: 1663
	Prompt Tokens: 1343
	Completion Tokens: 320
Successful Requests: 1
Total Cost (USD): $0.05949
Tokens Used: 1665
	Prompt Tokens: 1343
	Completion Tokens: 322
Successful Requests: 1
Total Cost (USD): $0.059609999999999996
Tokens Used: 1616
	Prompt Tokens: 1343
	Completion Tokens: 273
Successful Requests: 1
Total Cost (USD): $0.05667
Tokens Used: 1616
	Prompt Tokens: 1343
	Completion Tokens: 273
Successful Requests: 1
Total Cost (USD): $0.05667
Tokens Used: 1625
	Prompt Tokens: 1343
	Completion Tokens: 282
Successful Requests: 1
Total Cost (USD): $0.05721
Tokens Used: 1659
	Prompt Tokens: 1343
	Completion Tokens: 316
Successful Requests:

In [108]:
seed = 3656
train_data = 300

train_num = [6,8,10,20,30,40,50]


for i in train_num:
    df = pd.read_csv(f'../data/titanic/titanic_{seed}_{train_data}_train.csv').head(i)
    x_train = df.drop('survived', axis=1)
    y_train = df['survived']

    file_path = '../models/titanic/'
    for j in range(1,31):
        model = iblm.fit(x_train, y_train, model_name = f'titanic_{seed}_{i}_{j}', file_path=file_path)

Tokens Used: 2111
	Prompt Tokens: 1343
	Completion Tokens: 768
Successful Requests: 1
Total Cost (USD): $0.08637
Tokens Used: 1654
	Prompt Tokens: 1343
	Completion Tokens: 311
Successful Requests: 1
Total Cost (USD): $0.05895
Tokens Used: 1616
	Prompt Tokens: 1343
	Completion Tokens: 273
Successful Requests: 1
Total Cost (USD): $0.05667
Tokens Used: 1620
	Prompt Tokens: 1343
	Completion Tokens: 277
Successful Requests: 1
Total Cost (USD): $0.05691
Tokens Used: 1571
	Prompt Tokens: 1343
	Completion Tokens: 228
Successful Requests: 1
Total Cost (USD): $0.05397
Tokens Used: 1690
	Prompt Tokens: 1343
	Completion Tokens: 347
Successful Requests: 1
Total Cost (USD): $0.06111
Tokens Used: 1691
	Prompt Tokens: 1343
	Completion Tokens: 348
Successful Requests: 1
Total Cost (USD): $0.06117
Tokens Used: 2053
	Prompt Tokens: 1343
	Completion Tokens: 710
Successful Requests: 1
Total Cost (USD): $0.08288999999999999
Tokens Used: 1631
	Prompt Tokens: 1343
	Completion Tokens: 288
Successful Requests: 

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


Tokens Used: 3462
	Prompt Tokens: 3200
	Completion Tokens: 262
Successful Requests: 1
Total Cost (USD): $0.11172
Tokens Used: 3481
	Prompt Tokens: 3200
	Completion Tokens: 281
Successful Requests: 1
Total Cost (USD): $0.11286
Tokens Used: 3611
	Prompt Tokens: 3200
	Completion Tokens: 411
Successful Requests: 1
Total Cost (USD): $0.12066
Tokens Used: 3558
	Prompt Tokens: 3200
	Completion Tokens: 358
Successful Requests: 1
Total Cost (USD): $0.11748
Tokens Used: 3727
	Prompt Tokens: 3200
	Completion Tokens: 527
Successful Requests: 1
Total Cost (USD): $0.12762
Tokens Used: 3487
	Prompt Tokens: 3200
	Completion Tokens: 287
Successful Requests: 1
Total Cost (USD): $0.11322
Tokens Used: 3487
	Prompt Tokens: 3200
	Completion Tokens: 287
Successful Requests: 1
Total Cost (USD): $0.11322
Tokens Used: 3465
	Prompt Tokens: 3200
	Completion Tokens: 265
Successful Requests: 1
Total Cost (USD): $0.1119
Tokens Used: 3494
	Prompt Tokens: 3200
	Completion Tokens: 294
Successful Requests: 1
Total Cost 

In [109]:
seed = 3657
train_data = 300

train_num = [6,8,10,20,30,40,50]


for i in train_num:
    df = pd.read_csv(f'../data/titanic/titanic_{seed}_{train_data}_train.csv').head(i)
    x_train = df.drop('survived', axis=1)
    y_train = df['survived']

    file_path = '../models/titanic/'
    for j in range(1,31):
        model = iblm.fit(x_train, y_train, model_name = f'titanic_{seed}_{i}_{j}', file_path=file_path)

Tokens Used: 2519
	Prompt Tokens: 1342
	Completion Tokens: 1177
Successful Requests: 1
Total Cost (USD): $0.11087999999999998
Tokens Used: 1624
	Prompt Tokens: 1342
	Completion Tokens: 282
Successful Requests: 1
Total Cost (USD): $0.057179999999999995
Tokens Used: 1622
	Prompt Tokens: 1342
	Completion Tokens: 280
Successful Requests: 1
Total Cost (USD): $0.05706
Tokens Used: 1634
	Prompt Tokens: 1342
	Completion Tokens: 292
Successful Requests: 1
Total Cost (USD): $0.05778
Tokens Used: 1593
	Prompt Tokens: 1342
	Completion Tokens: 251
Successful Requests: 1
Total Cost (USD): $0.055319999999999994
Tokens Used: 1607
	Prompt Tokens: 1342
	Completion Tokens: 265
Successful Requests: 1
Total Cost (USD): $0.056159999999999995
Tokens Used: 1612
	Prompt Tokens: 1342
	Completion Tokens: 270
Successful Requests: 1
Total Cost (USD): $0.056459999999999996
Tokens Used: 1793
	Prompt Tokens: 1342
	Completion Tokens: 451
Successful Requests: 1
Total Cost (USD): $0.06731999999999999
Tokens Used: 1631
	

### Models Evaluation

In [99]:
import pandas as pd
import numpy as np
from langchain.llms import OpenAI
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from iblm import IBLMClassifier

import sys
import os

In [100]:
llm_model = OpenAI(temperature=0, model_name = 'gpt-4-0613')

params = {'columns_name': True}

iblm = IBLMClassifier(llm_model = llm_model, params=params)

In [115]:
seed = 3655
#seed = 3656
#seed = 3657


n = 300


train_num = [6,8,10,20,30,40,50]
i = 8


df = pd.read_csv(f'../data/titanic/titanic_{seed}_{n}_test.csv')
x_test = df.drop('survived', axis=1)
y_test = df['survived']

sys.path.append('..')

error_count = 0
error_list = []
auc_list = []
acc_list = []
model_list = []


for j in range(1,31):

    import_file = f'import models.titanic.titanic_{seed}_{i}_{j} as codemodel'


    try:
        exec(import_file)
        y_proba = codemodel.predict(x_test)
        y_pred = (y_proba > 0.5).astype(int)
        negative_values_exist = np.any(y_proba < 0)
        values_greater_than_one_exist = np.any(y_proba > 1)
        if negative_values_exist:
            error_list.append(j)
            error_count += 1
            print(f"Negative values exist：{negative_values_exist}")

        elif values_greater_than_one_exist:
            error_list.append(j)
            error_count += 1
            print(f"Positive values exist：{values_greater_than_one_exist}")

        else:
            roc_auc = roc_auc_score(y_test, y_proba)
            accuracy = round(accuracy_score(y_test, y_pred),4)
            auc_list.append(roc_auc)
            acc_list.append(accuracy)
            model_list.append(f'moon_{seed}_{i}_{j}')
        
    except Exception:
        print('run error')
        error_count += 1
        error_list.append(j)
        pass

print('-----------error-------------')
print(f'Number of errors:{error_count}')
print(error_list)

print('-----------Model-------------')
print(model_list)


print('-----------ACC-------------')
print(acc_list)
average = sum(acc_list) / len(acc_list)
max_value = max(acc_list)
min_value = min(acc_list)
print("Maximum Value:", round(max_value, 4))
print("Average Value:", round(average,4))
print("Minimum Value:", round(min_value, 4))


print('-----------AUC-------------')
print(auc_list)
average = sum(auc_list) / len(auc_list)
max_value = max(auc_list)
min_value = min(auc_list)
print("Maximum Value:", round(max_value,4))
print("Average Value:", round(average,4))
print("Minimum Value:", round(min_value,4))

run error
run error
run error
run error
run error
run error
run error
run error
-----------error-------------
Number of errors:8
[4, 16, 21, 25, 26, 27, 28, 29]
-----------Model-------------
['moon_3655_8_1', 'moon_3655_8_2', 'moon_3655_8_3', 'moon_3655_8_5', 'moon_3655_8_6', 'moon_3655_8_7', 'moon_3655_8_8', 'moon_3655_8_9', 'moon_3655_8_10', 'moon_3655_8_11', 'moon_3655_8_12', 'moon_3655_8_13', 'moon_3655_8_14', 'moon_3655_8_15', 'moon_3655_8_17', 'moon_3655_8_18', 'moon_3655_8_19', 'moon_3655_8_20', 'moon_3655_8_22', 'moon_3655_8_23', 'moon_3655_8_24', 'moon_3655_8_30']
-----------ACC-------------
[0.75, 0.75, 0.75, 0.74, 0.6767, 0.74, 0.6767, 0.4233, 0.67, 0.7333, 0.6767, 0.6767, 0.6767, 0.7333, 0.6767, 0.6767, 0.74, 0.6767, 0.5933, 0.6767, 0.7933, 0.7333]
Maximum Value: 0.7933
Average Value: 0.6927
Minimum Value: 0.4233
-----------AUC-------------
[0.8534248534248534, 0.7568997568997569, 0.8534248534248534, 0.8252538252538253, 0.5649220649220649, 0.8252538252538253, 0.564922064922